### Logistic Regression

In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 0)Prepare Data
bc=datasets.load_breast_cancer()
X,y = bc.data , bc.target

n_samples,n_features=X.shape

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=12345)


# Scale --> Convert Data to Zero Mean Variance 1
sc = StandardScaler()
X_train=sc.fit_transform(X_train) # Fit the scaler to the data and transform the data
X_test=sc.transform(X_test)

X_train=torch.from_numpy(X_train.astype(np.float32))
X_test=torch.from_numpy(X_test.astype(np.float32))
y_train=torch.from_numpy(y_train.astype(np.float32))
y_test=torch.from_numpy(y_test.astype(np.float32))

y_train=y_train.view(y_train.shape[0],1)
y_test=y_test.view(y_test.shape[0],1)



# 1)Model
# f = wx + b , sigmoid 
class LogisticRegression(nn.Module):
    def __init__(self,n_input_features):
        super(LogisticRegression,self).__init__()
        self.Linear=nn.Linear(n_input_features,1)

    def forward(self,x):
        y_predicted=torch.sigmoid(self.Linear(x))
        return y_predicted
    
model=LogisticRegression(n_features)

# 2)Loss and optimizer
learning_rate=0.01
criterion = nn.BCELoss() #Binary Cross Entropy Loss
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

# 3)Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    # Forward Pass and loss
    y_predicted=model(X_train)
    loss=criterion(y_predicted,y_train)

    # Backward pass
    loss.backward()

    # Update
    optimizer.step()

    # Zero Gradient
    optimizer.zero_grad()
    
    if (epoch+1)%10==0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

# Testing Model
with torch.no_grad():
    y_predicted=model(X_test)
    y_predicted_cls=y_predicted.round()
    acc=y_predicted_cls.eq(y_test).sum()/float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

 


epoch: 10, loss = 0.5156
epoch: 20, loss = 0.4350
epoch: 30, loss = 0.3819
epoch: 40, loss = 0.3441
epoch: 50, loss = 0.3156
epoch: 60, loss = 0.2933
epoch: 70, loss = 0.2752
epoch: 80, loss = 0.2601
epoch: 90, loss = 0.2474
epoch: 100, loss = 0.2365
accuracy = 0.9474
